# Exploración inicial de las variables

In [9]:
import pandas as pd
from pathlib import Path

DATA = Path("data") / "base_imputacion_mixta_1000.csv"
df = pd.read_csv(DATA)

df.head()


,fecha,sexo,ciudad,nivel_educativo,segmento,estado_civil,edad,altura_cm,ingresos,gasto_mensual,puntuacion_credito,demanda
0,2024-01-01,F,Medellín,NaN,B,Unión libre,19.0,161.821754,3574.753806,1832.731832,640.465372,119.202995
1,2024-01-02,F,Barranquilla,NaN,B,NaN,52.0,167.819566,3163.626815,NaN,533.108430,124.457874
2,2024-01-03,M,Bogotá,Secundaria,B,Soltero/a,38.0,165.756219,2765.672259,1219.535074,491.016910,NaN
3,2024-01-04,F,Bogotá,NaN,B,Casado/a,57.0,160.642670,4320.397345,1908.324816,NaN,129.426792
4,2024-01-05,M,Cali,Técnico,B,Soltero/a,67.0,151.402909,NaN,1887.385697,610.213994,133.916319


Cargamos la base `base_imputacion_mixta_1000.csv` con 1000 registros.  
La vista de las primeras filas nos permite confirmar la correcta lectura y empezar a reconocer variables.  
Podemos observar columnas como `sexo`, `ciudad`, `edad`, `ingresos`, `puntuacion_credito`, entre otras, que mezclan datos **cualitativos** y **cuantitativos**.


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fecha               1000 non-null   object 
 1   sexo                980 non-null    object 
 2   ciudad              950 non-null    object 
 3   nivel_educativo     900 non-null    object 
 4   segmento            800 non-null    object 
 5   estado_civil        650 non-null    object 
 6   edad                970 non-null    float64
 7   altura_cm           920 non-null    float64
 8   ingresos            880 non-null    float64
 9   gasto_mensual       750 non-null    float64
 10  puntuacion_credito  500 non-null    float64
 11  demanda             850 non-null    float64
dtypes: float64(6), object(6)
memory usage: 93.9+ KB


El método `info()` muestra la estructura del dataset:  
- **Número de filas y columnas:** 1000 observaciones y 12 variables.  
- **Tipos de datos:** mezcla de variables categóricas (`object`) y numéricas (`float64`, `int64`).  
- **Valores nulos:** se observan ausencias en varias columnas, especialmente en `puntuacion_credito` (50% faltante).  
Esto confirma que necesitaremos técnicas de imputación distintas para cada tipo de variable.


In [5]:
df.describe(include="all").T


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
fecha,1000,1000,2024-01-01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sexo,980,2,F,518,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ciudad,950,5,Bogotá,307,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nivel_educativo,900,4,Secundaria,317,NaN,NaN,NaN,NaN,NaN,NaN,NaN
segmento,800,3,B,457,NaN,NaN,NaN,NaN,NaN,NaN,NaN
estado_civil,650,4,Soltero/a,290,NaN,NaN,NaN,NaN,NaN,NaN,NaN
edad,970.0,NaN,NaN,NaN,42.861856,14.621382,18.0,30.0,43.0,55.0,69.0
altura_cm,920.0,NaN,NaN,NaN,167.760096,9.27553,140.0,161.488768,167.714614,173.999069,195.766921
ingresos,880.0,NaN,NaN,NaN,3681.294745,1079.326096,487.662547,2999.416229,3669.620507,4375.093656,7016.246936
gasto_mensual,750.0,NaN,NaN,NaN,1687.810749,582.070174,100.0,1309.239768,1676.193764,2063.26099,3532.593603


El resumen estadístico muestra información clave:  
- Para las **numéricas**: valores mínimos, máximos, medias y desviaciones estándar.  
  - La edad va de 18 a 69 años, con media ≈ 42.8.  
  - Los ingresos van de 487 a 7016, mostrando alta dispersión.  
  - La puntuación de crédito varía entre 374 y 823, pero con 50% de nulos.  
- Para las **categóricas**: número de categorías distintas y la categoría más frecuente.  
  - `ciudad` tiene 5 categorías (Bogotá es la dominante con 307 casos).  
  - `nivel_educativo` tiene 4 categorías (Secundaria es la más común).  
  - `segmento` se concentra en la categoría B.  

Esto nos da una idea clara de la distribución y relevancia de cada variable.


In [6]:
numericas = df.select_dtypes(include=["number"]).columns.tolist()
categoricas = df.select_dtypes(exclude=["number"]).columns.tolist()

print("Variables numéricas:", numericas)
print("Variables categóricas:", categoricas)


Variables numéricas: ['edad', 'altura_cm', 'ingresos', 'gasto_mensual', 'puntuacion_credito', 'demanda']
Variables categóricas: ['fecha', 'sexo', 'ciudad', 'nivel_educativo', 'segmento', 'estado_civil']


Las variables **numéricas** incluyen: `edad`, `altura_cm`, `ingresos`, `gasto_mensual`, `puntuacion_credito`, `demanda`.  
Las variables **categóricas** incluyen: `sexo`, `ciudad`, `nivel_educativo`, `segmento`, `estado_civil`.  
Además, `fecha` es categórica pero actúa más como un identificador temporal que como variable de análisis.  

Esta separación es fundamental porque la imputación y análisis posterior dependerán del tipo de variable.
